# Evaluate LBPH Face Recognition Model

In [1]:
import cv2
import os
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, log_loss

## Function to Read Images and Labels

In [2]:
def read_images_and_labels(dataset_path, img_size=(200, 200)):
    images = []
    labels = []
    label_map = {}
    label_counter = 0

    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_path):
            continue

        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue

            # Resize the image to ensure all images have the same dimensions
            image = cv2.resize(image, img_size)

            if person_name not in label_map:
                label_map[person_name] = label_counter
                label_counter += 1

            label = label_map[person_name]
            images.append(image)
            labels.append(label)

    return images, labels, label_map

## Load the Recognizer Model and Label Map

In [3]:
# Load the trained model
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('lbph_face_recognizer_model.yml')

# Load the label map
with open('label_map.pkl', 'rb') as f:
    label_map = pickle.load(f)

# Invert the label map
label_map_inv = {v: k for k, v in label_map.items()}

## Function to Evaluate the Model

In [22]:
def mod_log_loss(y_true, y_pred, labels):
	class_diff = labels - y_pred[0]

	if(class_diff > 0):
		y_pred_pad = np.array([np.pad(x, pad_width=(0,class_diff)) for x in y_pred])
	else:
		y_pred_pad = y_pred

	return(log_loss(y_true, y_pred_pad, labels=labels))

def evaluate_model(images, labels, num_classes):
    correct_predictions = 0
    log_loss_sum = 0
    epsilon = 1e-15  # Small value to prevent log(0)
    
    true_labels = []
    pred_labels = []

    for i, image in enumerate(images):
        label, confidence = recognizer.predict(image)
        correct_predictions += (label == labels[i])

        true_labels.append(labels[i])
        pred_labels.append(label)

        # Simulate probability distributions for custom log loss
        predicted_prob = np.ones(num_classes) * (1 - confidence) / (num_classes - 1)
        predicted_prob[label] = confidence
        predicted_prob = np.clip(predicted_prob, epsilon, 1 - epsilon)
        predicted_prob /= predicted_prob.sum()

        true_prob = np.zeros(num_classes)
        true_prob[labels[i]] = 1

        log_loss_sum += -np.sum(true_prob * np.log(predicted_prob))

    accuracy = accuracy_score(true_labels, pred_labels)
    entropy_loss = mod_log_loss(true_labels, pred_labels, labels=range(num_classes))
    return accuracy, entropy_loss

## Read Validation Dataset

In [23]:
# Specify the path to your validation dataset
validation_dataset_path = "dataset2"

# Read validation images and labels
val_images, val_labels, _ = read_images_and_labels(validation_dataset_path)

## Evaluate the Model

In [24]:
# Get the number of unique classes from the label map
num_classes = len(label_map)

# Evaluate the model
val_accuracy, val_loss = evaluate_model(val_images, val_labels, num_classes)

# Print the evaluation results
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss (Entropy): {val_loss:.4f}")

TypeError: unsupported operand type(s) for -: 'range' and 'int'